In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
for name, value in os.environ.items():
    print("{0}: {1}".format(name, value))

PATH: /opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME: b8308b43e46e
TERM: xterm-color
DEBIAN_FRONTEND: noninteractive
CONDA_DIR: /opt/conda
SHELL: /bin/bash
NB_USER: jovyan
NB_UID: 1000
NB_GID: 100
LC_ALL: en_US.UTF-8
LANG: en_US.UTF-8
LANGUAGE: en_US.UTF-8
HOME: /home/jovyan
JUPYTER_PORT: 8888
PYDEVD_USE_FRAME_EVAL: NO
JPY_SESSION_NAME: /perpetual/notebooks/google_cvrp.ipynb
JPY_PARENT_PID: 7
CLICOLOR: 1
FORCE_COLOR: 1
CLICOLOR_FORCE: 1
PAGER: cat
GIT_PAGER: cat
MPLBACKEND: module://matplotlib_inline.backend_inline
path_to_dataframe: ../data/outputs/notebook_output_combined_dropoff_pickup_toy.csv
path_to_distance_matrix: ../data/outputs/notebook_output_combined_dropoff_pickup_dists_toy.csv
num_vehicles: 2
output_path: ../data/outputs/routes/converted_truck_dropoff_galv
vehicle_capacity: 120
num_seconds_simulation: 100
capacity: Capacity
depot_index: 0


In [4]:
"""
This notebook is a port of Fall 2023's implementation of 
Google ORTools' Capacited Vehicles Routing Problem
(CVRP). The script was used to determine the optimal routing scheme for
our problem.

1. set your arguments in the pipeline/utils/config_inputs.ini file
under [optimize google cvrp]

2. Run this script in the terminal using:
cd code
python optimize_cvrp.py
"""

import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [5]:
def get_demands(location_df, capacity):
    """
    This function will get the daily number of totes to be
    picked up at every location provided in the df.

    Inputs: location_df = df of all the service locations
                        on the daily truck route
    Outputs: demands_list = a list of the number of totes to be collected
                            at each location daily
                            (in the same order of locations from the df)
    """
    demands_list = []
    for index, row in location_df.iterrows():
        demands_list.append(int(row[capacity]))

    return demands_list


def create_data_model(
    path_locations_df,
    path_distance_matrix,
    num_vehicles,
    vehicle_capacity,
    capacity,
    depot_index,
):
    """Stores the data for the problem."""
    data = {}
    locations_df = pd.read_csv(path_locations_df)
    distance_matrix = pd.read_csv(path_distance_matrix)

    data["distance_matrix"] = distance_matrix.to_numpy().astype(int)
    data["demands"] = get_demands(locations_df, capacity)
    data["num_vehicles"] = num_vehicles
    capacity = vehicle_capacity
    data["vehicle_capacities"] = [capacity for i in range(data["num_vehicles"])]
    data["depot"] = depot_index
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} \
            Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def save_to_table(data, manager, routing, solution):
    """Save each route to its own dataframe
    and print solutions on the console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    routes = []
    distances = []
    loads = []
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        route = []
        agg_distances = []
        truck_load = []
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            # route.append({node_index:route_load})
            route.append(node_index)
            truck_load.append(route_load)
            agg_distances.append(route_distance)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )

        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        route.append(manager.IndexToNode(index))
        truck_load.append(route_load)
        agg_distances.append(route_distance)
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
        routes.append(route)
        distances.append(agg_distances)
        loads.append(truck_load)
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")
    return routes, distances, loads


def make_dataframe(
    path_locations_df, output_path, data, manager, routing, solution
):
    """use the output of save_to_table to save the dataframe as a
    csv file in the data folder"""
    locations_df = pd.read_csv(path_locations_df)
    routes, distances, loads = save_to_table(data, manager, routing, solution)
    for i in range(len(routes)):
        route_df = locations_df.loc[routes[i], :]
        route_df["Cumulative_Distance"] = distances[i]
        route_df["Truck_Load"] = loads[i]
        route_df = route_df.reset_index()
        route_df = route_df.rename(columns={"index": "Original_Index"})

        path = output_path + "/route" + str(i + 1) + ".csv"
        route_df.to_csv(path, index=False)


def solve_and_save(
    path_locations_df,
    path_distance_matrix,
    num_vehicles,
    vehicle_capacity,
    num_seconds,
    capacity,
    depot_index,
    output_path,
):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(
        path_locations_df,
        path_distance_matrix,
        num_vehicles,
        vehicle_capacity,
        capacity,
        depot_index,
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback
    )
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(num_seconds)
    # search_parameters.time_limit.seconds = 7200

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Return solution.
    if solution:
        # print_solution(data, manager, routing, solution)
        make_dataframe(
            path_locations_df, output_path, data, manager, routing, solution
        )


In [6]:
# read .env variables
path_locations_df = os.environ.get("path_to_dataframe")
path_distance_matrix = os.environ.get("path_to_distance_matrix")
num_vehicles = int(os.environ.get("num_vehicles"))
vehicle_capacity = int(os.environ.get("vehicle_capacity"))
num_seconds = int(os.environ.get("num_seconds_simulation"))
capacity = os.environ.get("capacity")
depot_index = int(os.environ.get("depot_index"))
output_path = os.environ.get("output_path")

In [8]:
solve_and_save(
    path_locations_df=path_locations_df,
    path_distance_matrix=path_distance_matrix,
    num_vehicles=num_vehicles,
    vehicle_capacity=vehicle_capacity,
    num_seconds=num_seconds,
    capacity=capacity,
    depot_index=depot_index,
    output_path=output_path,
)

Objective: 17
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  1 Load(1) ->  2 Load(0) ->  3 Load(1) ->  4 Load(0) ->  5 Load(1) ->  6 Load(0) ->  7 Load(1) ->  8 Load(0) ->  9 Load(1) ->  10 Load(0) ->  11 Load(1) ->  12 Load(0) ->  13 Load(1) ->  14 Load(0) ->  15 Load(1) ->  16 Load(0) ->  17 Load(1) ->  18 Load(0) ->  19 Load(1) ->  20 Load(0) ->  21 Load(1) ->  22 Load(0) ->  23 Load(1) ->  24 Load(0) ->  25 Load(1) ->  26 Load(0) ->  27 Load(1) ->  28 Load(0) ->  29 Load(1) ->  30 Load(0) ->  31 Load(1) ->  32 Load(0) ->  33 Load(3) ->  34 Load(1) ->  0 Load(1)
Distance of the route: 17m
Load of the route: 1

Total distance of all routes: 17m
Total load of all routes: 1
